In [76]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    NormalizedString,
    Regex,
)
from tokenizers.normalizers import Normalizer
import re

In [ ]:
tokenizer = Tokenizer(models.BPE)

# Normalizers

1. Unicode normalization 
2. Mapping from 'y' to 'i (bác sỹ -> bác sĩ) except when 'y' is alone or when it's in the compound 'uy' 

In [153]:
class CustomNormalizer: 
    def __init__(self): 
        self.pattern = re.compile(r" [hklmnst][^a\á\à\ạ\ả\ã\â\ấ\ầ\ậ\ẩ\ẫ\ă\ắ\ằ\ẳ\ẵ\ặu\s]*[y\ỷ\ỹ\ỵ\ỳ\ý]{1}\W{1}")
        self.y_to_i_map = {
            'y': 'i',
            'ỷ': 'ỉ',
            'ỹ': 'ĩ',
            'ỵ': 'ị',
            'ỳ': 'ì',
            'ý': 'í'
        }

    def normalize(self, normalized: NormalizedString): 
        original = normalized.original
        matches = self.pattern.findall(original)

        for match in matches: 
            replacement = match[0:-2:1] + self.y_to_i_map[match[-2]] + match[-1]
            normalized.replace(match, replacement)

        print(normalized.normalized)
        normalized.nfc()

In [154]:
custom_normalizer = Normalizer.custom(CustomNormalizer())

In [156]:
test_text = "Tôi là một người bác sỹ và ca sỹ, tôi bị suy."
norm = NormalizedString(test_text)
norm.lowercase()

norm.normalized

'tôi là một người bác sỹ và ca sỹ, tôi bị suy.'

In [157]:
normalized_res = custom_normalizer.normalize(norm)

tôi là một người bác sĩ và ca sĩ, tôi bị suy.


# Pre-tokenizers


1. Add whitespace prefix 